In [5]:
pip install geopandas

   ---------------------------------------- 0.0/19.2 MB ? eta -:--:--
   - -------------------------------------- 0.5/19.2 MB 1.5 MB/s eta 0:00:13
   ---- ----------------------------------- 2.4/19.2 MB 5.2 MB/s eta 0:00:04
   ---------- ----------------------------- 5.0/19.2 MB 7.9 MB/s eta 0:00:02
   ---------------- ----------------------- 8.1/19.2 MB 9.5 MB/s eta 0:00:02
   ----------------------- ---------------- 11.3/19.2 MB 10.5 MB/s eta 0:00:01
   ------------------------------ --------- 14.4/19.2 MB 11.2 MB/s eta 0:00:01
   ------------------------------------- -- 17.8/19.2 MB 11.8 MB/s eta 0:00:01
   ---------------------------------------- 19.2/19.2 MB 11.5 MB/s  0:00:01
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ------------- -------------------------- 2.1/6.3 MB 10.5 MB/s eta 0:00:01
   ---------------------------- ----------- 4.5/6.3 MB 10.9 MB/s eta 0:00:01
   ---------------------------------------  6.3/6.3 MB 10.9 MB/s eta 0:00:01
   ------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
pip install reverse_geocoder

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd

dc = pd.read_csv("Data_Centers_Database - Data Centers.csv")


dc.columns = dc.columns.str.strip().str.lower().str.replace(" ", "_")
dc = dc.dropna(subset=["lat", "long"])
dc.to_csv("data_centers_clean.csv", index=False)



In [ ]:
import pandas as pd


epa = pd.read_csv("annual_conc_by_monitor_2022.csv")


keep_pollutants = [
    "PM2.5 - Local Conditions",
    "PM2.5 - Local Conditions (1st field if there are variations)",
    "Ozone",
    "Nitrogen dioxide (NO2)"
] 
epa = epa[epa["Parameter Name"].isin(keep_pollutants)]

cols_to_keep = [
    "Latitude", "Longitude",
    "State Code", "County Code",
    "State Name", "County Name", "City Name",
    "Parameter Name", "Parameter Code",
    "Year",
    "Arithmetic Mean",
    "99th Percentile",
    "Local Site Name"
]
epa_clean = epa[cols_to_keep].copy()

epa_clean = epa_clean.dropna(subset=["Latitude", "Longitude", "Arithmetic Mean"])

epa_clean.columns = (
    epa_clean.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
)

epa_clean.to_csv("air_quality_clean.csv", index=False)


In [ ]:

dc["lat"] = (
    dc["lat"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .astype(float)
)

dc["long"] = (
    dc["long"]
    .astype(str)
    .str.replace(",", "", regex=False)
    .str.strip()
    .astype(float)
)


In [ ]:

dc["lat"] = (
    dc["lat"].astype(str)
             .str.replace(",", "", regex=False)
             .str.replace("°", "", regex=False)
             .str.strip()
)

dc["long"] = (
    dc["long"].astype(str)
             .str.replace(",", "", regex=False)
             .str.replace("°", "", regex=False)
             .str.strip()
)


dc["lat"] = dc["lat"].str.extract(r"([-+]?\d*\.\d+|\d+)").astype(float)
dc["long"] = dc["long"].str.extract(r"([-+]?\d*\.\d+|\d+)").astype(float)



In [ ]:
import reverse_geocoder as rg

coords = list(zip(dc["lat"], dc["long"]))

results = rg.search(coords)


dc["state"] = [r["admin1"] for r in results]
dc["county"] = [r["admin2"] for r in results]


dc.to_csv("data_centers_geo_enriched.csv", index=False)


In [22]:
epa = pd.read_csv("air_quality_clean.csv")
epa.head()
epa_grouped = (
    epa.groupby(["state_name", "county_name", "parameter_name"])
       .agg(
            mean_value=("arithmetic_mean", "mean"),
            max_99th=("99th_percentile", "mean")
        )
       .reset_index()
)
epa_pivot = epa_grouped.pivot_table(
    index=["state_name", "county_name"],
    columns="parameter_name",
    values="mean_value"
).reset_index()


In [25]:
epa_pivot.columns = (
    epa_pivot.columns
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace(")", "")
    .str.replace("-", "")
)
epa_pivot.head()

parameter_name,state_name,county_name,nitrogen_dioxide_no2,ozone,pm2.5__local_conditions
0,Alabama,Baldwin,NaN,0.040524,7.317391
1,Alabama,Clay,NaN,NaN,6.880870
2,Alabama,DeKalb,NaN,0.042298,6.855932
3,Alabama,Elmore,NaN,0.037575,NaN
4,Alabama,Etowah,NaN,0.042096,8.231535


In [26]:
merged = dc.merge(
    epa_pivot,
    left_on=["state", "county"],
    right_on=["state_name", "county_name"],
    how="left"
)


In [27]:
merged.to_csv("dc_with_air_quality.csv", index=False)
